In [14]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")


def recuData():  
    link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
    data = pd.read_excel(link,'TODO CORREGIMIENTO')
    return data

def cleanBefore(dt):  
    data = dt
    # eliminamos filas que no importan y asignamos la primera como cabecera
    data.drop([0], inplace=True)
    data.columns = data.iloc[0]
    data.drop([1], inplace=True)
    #eliminamos la primera fila que es un NA
    our_data_filter = data.iloc[1:]
    # extraemos un subconjunto que llegue solo hasta el dia de hoy:
    now = datetime.datetime.now()
    our_data_filter = our_data_filter[our_data_filter['Fecha'] < now]
    # filtramos desde el primer dia en el que haya recuperados:
    our_data_filter = our_data_filter[our_data_filter['NuevosRecuperadosacum'] > 0]
    # extraemos solo las columnas que nos interesan:
    our_data_filter = our_data_filter[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]
    return our_data_filter

def transAAC(data_transAAC):
    data_transAAC.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)
    data_transAAC['NuevosConfirmados'] = data_transAAC.to_numeric(data_transAAC['NuevosConfirmados'])
    data_transAAC['NuevosFallecidos'] = data_transAAC.to_numeric(data_transAAC['NuevosFallecidos'])
    data_transAAC['NuevosRecuperados'] = data_transAAC.to_numeric(data_transAAC['NuevosRecuperados'])
    data_transAAC['Poblacion'] = data_transAAC.to_numeric(data_transAAC['Poblacion'])
    data_transAAC['NuevosConfirmados'] = data_transAAC['NuevosConfirmados']+data_transAAC['NuevosFallecidos']
    data_transAAC['NuevosConfirmadosacum'] = data_transAAC['NuevosConfirmados'].cumsum()
    data_transAAC['NuevosRecuperadosacum'] = data_transAAC['NuevosRecuperados'].cumsum()
    data_transAAC["Susceptibles"] = data_transAAC['Poblacion']-data_transAAC['NuevosConfirmadosacum']-data_transAAC['NuevosRecuperadosacum']
    data_transAAC["pob_log_Susceptibles"] = data_transAAC['Poblacion']*np.log(data_transAAC["Susceptibles"])
    return data_transAAC


def calculate(data_cal):
    lista_R0 = []
    y = our_data_filter_un_final["pob_log_Susceptibles"]
    for i in range(len(y)):
        try:
            y = our_data_filter_un_final["pob_log_Susceptibles"]
            x = our_data_filter_un_final["NuevosRecuperadosacum"]
            yy = y.head(i+1)
            xx = x.head(i+1)
        # Creamos el objeto de Regresión Lineal
            regresion_lineal = linear_model.LinearRegression()
            regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
            r0 = -regresion_lineal.coef_
            lista_R0.append(r0)       
        except:
            print("Error")
            lista_R0.append(0)      
    our_data_filter_un_final["r0"] = lista_R0
    return our_data_filter_un_final

def cleanAfter(data_second_r0):
    x = data_second_r0['r0'].str.replace("]]", "")
    y = x.str.replace("[", "")
    z_dataframe = pd.DataFrame(z)
    del data_second_r0['r0']
    data_second_r0['r0'] = z_dataframe
    return data_second_r0

def saveParcial(dt):
    #Dejar los generados archivos en la misma carpeta
    dt.to_csv('test.csv', mode='a', index = False, header=False)
    return 

def R0Panama():
    
    
    
    data = recuData()
    
    CD_Corr_unique = np.unique(data['CD_Corr'])
    cantidad_de_coor = len(CD_Corr_unique)
    
    for i in range(cantidad_de_coor):

        our_data_filter = our_data[our_data['CD_Corr'] == CD_Corr_unique[i]]

    return




In [18]:
data = recuData()
data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,2020-08-15 00:00:00,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ON,Ultimo,Indice,CD_Corr,Provincia,Distrito,Corregimiento,CD_Prov,Fecha,DIA,...,%Activos,%Nuevos Casos,%Nuevos Fallecidos,%Nuevos Recuperados,%Nuevos Activos,Casos/1MM hab,Fallecidos/1MM hab,Recuperados/1 MM hab,Activos/1MM hab,Población
2,ON,no,1,80817,Panamá,Panamá,Pacora,1,2020-06-05 00:00:00,1,...,0.393728,NaN,NaN,NaN,NaN,16373.8,456.413,9470.56,6446.83,35056
3,ON,no,2,80819,Panamá,Panamá,Tocumen,2,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,91447
4,ON,no,3,130101,Panamá Oeste,Arraiján,Arraiján (Cabecera),3,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,53634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70767,OUT TIME,no,670,70314,Los Santos,Los Santos,Villa Lourdes,7,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,1088
70768,OUT TIME,no,671,20107,Coclé,Aguadulce,Virgen del Carmen,8,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,10367
70769,OUT TIME,no,672,50206,Darién,Pinogana,Yape,9,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,216
70770,OUT TIME,no,673,50317,Darién,Santa Fe,Zapallal,10,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,2072


In [17]:
data

1,ON,Ultimo,Indice,CD_Corr,Provincia,Distrito,Corregimiento,CD_Prov,Fecha,DIA,...,%Activos,%Nuevos Casos,%Nuevos Fallecidos,%Nuevos Recuperados,%Nuevos Activos,Casos/1MM hab,Fallecidos/1MM hab,Recuperados/1 MM hab,Activos/1MM hab,Poblacion
2,ON,no,1,80817,Panamá,Panamá,Pacora,1,2020-06-05 00:00:00,1,...,0.393728,NaN,NaN,NaN,NaN,16373.8,456.413,9470.56,6446.83,35056
3,ON,no,2,80819,Panamá,Panamá,Tocumen,2,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,91447
4,ON,no,3,130101,Panamá Oeste,Arraiján,Arraiján (Cabecera),3,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,53634
5,ON,no,4,80821,Panamá,Panamá,24 de Diciembre,4,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,79798
6,ON,no,5,81002,Panamá,San Miguelito,Belisario Porras,5,2020-06-05 00:00:00,1,...,1,NaN,NaN,NaN,NaN,17.8728,0,0,17.8728,55951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70767,OUT TIME,no,670,70314,Los Santos,Los Santos,Villa Lourdes,7,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,1088
70768,OUT TIME,no,671,20107,Coclé,Aguadulce,Virgen del Carmen,8,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,10367
70769,OUT TIME,no,672,50206,Darién,Pinogana,Yape,9,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,216
70770,OUT TIME,no,673,50317,Darién,Santa Fe,Zapallal,10,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,2072


In [16]:

data2 = transAAC(data)

data2

AttributeError: 'DataFrame' object has no attribute 'to_numeric'

In [5]:
data['CD_Corr'].unique()

KeyError: 'CD_Corr'

In [19]:
def recuData():  
    link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
    data = pd.read_excel(link,'TODO CORREGIMIENTO')
    return data
recuData()
data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,2020-08-15 00:00:00,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ON,Ultimo,Indice,CD_Corr,Provincia,Distrito,Corregimiento,CD_Prov,Fecha,DIA,...,%Activos,%Nuevos Casos,%Nuevos Fallecidos,%Nuevos Recuperados,%Nuevos Activos,Casos/1MM hab,Fallecidos/1MM hab,Recuperados/1 MM hab,Activos/1MM hab,Población
2,ON,no,1,80817,Panamá,Panamá,Pacora,1,2020-06-05 00:00:00,1,...,0.393728,NaN,NaN,NaN,NaN,16373.8,456.413,9470.56,6446.83,35056
3,ON,no,2,80819,Panamá,Panamá,Tocumen,2,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,91447
4,ON,no,3,130101,Panamá Oeste,Arraiján,Arraiján (Cabecera),3,2020-06-05 00:00:00,1,...,0,NaN,NaN,NaN,NaN,0,0,0,0,53634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70767,OUT TIME,no,670,70314,Los Santos,Los Santos,Villa Lourdes,7,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,1088
70768,OUT TIME,no,671,20107,Coclé,Aguadulce,Virgen del Carmen,8,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,10367
70769,OUT TIME,no,672,50206,Darién,Pinogana,Yape,9,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,216
70770,OUT TIME,no,673,50317,Darién,Santa Fe,Zapallal,10,2020-09-17 00:00:00,105,...,0,0,0,0,0,0,0,0,0,2072
